In [2]:
import pandas as pd
import numpy as np

In [3]:
df_bt = pd.read_csv('bt_score.txt', header=0, sep='\t')
df_jt = pd.read_csv('jt_score.txt', header=0, sep='\t')

In [5]:
df_bt['dt'].value_counts().index.sort_values()

Index(['2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05',
       '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10',
       '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15',
       '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20',
       '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25',
       '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-05-30',
       '2020-05-31', '2020-06-09', '2020-06-10', '2020-06-11', '2020-06-12',
       '2020-06-13', '2020-06-14', '2020-06-15', '2020-06-16', '2020-06-17',
       '2020-06-18', '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22',
       '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26'],
      dtype='object')

In [6]:
date_list1 = ['2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05',
       '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10',
       '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15',
       '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20',
       '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25',
       '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-05-30',
                     '2020-06-09', '2020-06-10', '2020-06-11', '2020-06-12',
       '2020-06-13', '2020-06-14', '2020-06-15', '2020-06-16', '2020-06-17',
       '2020-06-18', '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22',
       '2020-06-23', '2020-06-24', '2020-06-25'              ]
date_list2 = [              '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05',
       '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10',
       '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15',
       '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20',
       '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25',
       '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-05-30',
       '2020-05-31',               '2020-06-10', '2020-06-11', '2020-06-12',
       '2020-06-13', '2020-06-14', '2020-06-15', '2020-06-16', '2020-06-17',
       '2020-06-18', '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22',
       '2020-06-23', '2020-06-24', '2020-06-25', '2020-06-26']

In [7]:
def Cal_Psi(score, pre_score, length=10):
    import math
    labels=['c'+str(i) for i in range(length)]
    True_out,bins=pd.qcut(score,q=length,retbins=True,labels=labels)
    bins[0] = bins[0]-0.0001 #cut左开右闭，之前最小值再分组后组记号为空，这里减0.0001划到最左侧区间
    
    Pre_out,bins_=pd.cut(pre_score,bins=bins,retbins=True,labels=labels)
    #print(bins_)
    
    a=pd.DataFrame(pd.Series(True_out).value_counts()).rename(columns={0:'val1'})
    a=a.applymap(lambda y : y/len(score)) #计算实际值区间占比
    #print(a)
    b=pd.DataFrame(pd.Series(Pre_out).value_counts()).rename(columns={0:'val2'})
    b=b.applymap(lambda y : y/len(score))  #计算预测值区间占比
    #print(b)
    re=pd.merge(a,b,left_index=True,right_index=True)
    #print(re)
    psi=0
    for i in range(len(re)):
        if re['val1'][i]==0:
            re['val1'][i]=0.000001
        if re['val2'][i]==0:
            re['val2'][i]=0.000001
        p=((re['val2'][i]-re['val1'][i])*(math.log((re['val2'][i]/re['val1'][i]))))
        
        psi=psi+p
    return psi

In [20]:
score = [1,2,3,4,5,6,7,8,8,8,8]
pre_score = [2,3,4,5,6,7,8,5,5,5,5]
Cal_Psi(score, pre_score, length=2)

[0.9999 6.     8.    ]
        val1
c0  0.545455
c1  0.454545
        val2
c0  0.818182
c1  0.181818
        val1      val2
c0  0.545455  0.818182
c1  0.454545  0.181818


0.3604788654497235

In [8]:
#初始化字典
dict_bt_org_score = {}
for i in range(len(df_bt)):
    dict_bt_org_score[df_bt.iloc[i,0]] = []
#填充字典
for i in range(len(df_bt)):
    dict_bt_org_score[df_bt.iloc[i,0]].append(df_bt.iloc[i,1])
#计算PSI
res_bt_org_score = {}
for index, date in enumerate(date_list1):
    res_bt_org_score[date] = Cal_Psi(dict_bt_org_score[date], dict_bt_org_score[date_list2[index]], length=10)

In [ ]:
#初始化字典
dict_bt_org_score_cuiji = {}
for i in range(len(df_bt)):
    dict_bt_org_score_cuiji[df_bt.iloc[i,0]] = []
#填充字典
for i in range(len(df_bt)):
    dict_bt_org_score_cuiji[df_bt.iloc[i,0]].append(df_bt.iloc[i,2])
#计算PSI
res_bt_org_score_cuiji = {}
for index, date in enumerate(date_list1):
    res_bt_org_score_cuiji[date] = Cal_Psi(dict_bt_org_score_cuiji[date], dict_bt_org_score_cuiji[date_list2[index]], length=10)

In [ ]:
#初始化字典
dict_jt_org_score = {}
for i in range(len(df_jt)):
    dict_jt_org_score[df_jt.iloc[i,0]] = []
#填充字典
for i in range(len(df_jt)):
    dict_jt_org_score[df_jt.iloc[i,0]].append(df_jt.iloc[i,1])
#计算PSI
res_jt_org_score = {}
for index, date in enumerate(date_list1):
    res_jt_org_score[date] = Cal_Psi(dict_jt_org_score[date], dict_jt_org_score[date_list2[index]], length=10)

In [11]:
#初始化字典
dict_jt_org_score_cuiji = {}
for i in range(len(df_jt)):
    dict_jt_org_score_cuiji[df_jt.iloc[i,0]] = []
#填充字典
for i in range(len(df_jt)):
    dict_jt_org_score_cuiji[df_jt.iloc[i,0]].append(df_jt.iloc[i,2])
#计算PSI
res_jt_org_score_cuiji = {}
for index, date in enumerate(date_list1):
    res_jt_org_score_cuiji[date] = Cal_Psi(dict_jt_org_score_cuiji[date], dict_jt_org_score_cuiji[date_list2[index]], length=10)

In [12]:
pd.DataFrame(list(res_bt_org_score.items())).to_csv('bt_org_score.txt', sep = '\t')
pd.DataFrame(list(res_bt_org_score_cuiji.items())).to_csv('bt_org_score_cuiji.txt', sep = '\t')
pd.DataFrame(list(res_jt_org_score.items())).to_csv('jt_org_score.txt', sep = '\t')
pd.DataFrame(list(res_jt_org_score_cuiji.items())).to_csv('jt_org_score_cuiji.txt', sep = '\t')